#News paper categorization using Machine Learning models
There are 4 categories in it which are:

Politics: 0 

Technology: 1 

Entertainment:2 

Business: 3

There are 3 excels sheets given for categorization train, test and sample submission

In [93]:
#importing basic libraries
import pandas as pd 
import numpy as np 

In [94]:
from google.colab import drive #for mounting the google drive 

In [101]:
#locating current working directory where our data is stored
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
cd /content/drive/"My Drive/Colab Notebooks/data1"

/content/drive/My Drive/Colab Notebooks/data1


In [103]:
#load data
df_train = pd.read_excel('Data_Train.xlsx')
df_test = pd.read_excel('Data_Test.xlsx')
df_sample = pd.read_excel('Sample_submission.xlsx')

In [104]:
#print first 5 data from train excel
df_train.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [105]:
#print first 5 data from test excel
df_test.head()

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [106]:
#print first 5 data from submission example excel
df_sample.head()

,SECTION
0,3
1,3
2,3
3,3
4,3


Data cleaning and preprocessing

In [107]:
#making function for removing noise from the data
import re #regular expression library for combine a regular expression pattern into pattern objects, which is used for pattern matching.
import itertools #worls like for loop it works ast as compare to for loop
def cleanData(clean_data):
    # Make all data in lowercase
    clean_data = clean_data.apply(lambda x: " ".join(x.lower() for x in x.split()))
    # Remove whitespaces from sentences
    clean_data = clean_data.apply(lambda x: " ".join(x.strip() for x in x.split()))
    # Convert data to string
    clean_data = clean_data.astype(str)
    return clean_data

In [108]:
cleanData

<function __main__.cleanData>

In [109]:
#cleanData called for removinf whitespace, noise, converting into lowercase and string
df_train['STORY'] = cleanData(df_train['STORY'])
df_test['STORY'] = cleanData(df_test['STORY'])
#in sample data there is no story column so we don't need 

In [110]:
df_train['STORY']

0       but the most painful was the huge reversal in ...
1       how formidable is the opposition alliance amon...
2       most asian currencies were trading lower today...
3       if you want to answer any question, click on ‘...
4       in global markets, gold prices edged up today ...
                              ...                        
7623    karnataka has been a congress bastion, but it ...
7624    the film, which also features janhvi kapoor, w...
7625    the database has been created after bringing t...
7626    the state, which has had an uneasy relationshi...
7627    virus stars kunchacko boban, tovino thomas, in...
Name: STORY, Length: 7628, dtype: object

In [111]:
df_test['STORY']

0       2019 will see gadgets like gaming smartphones ...
1       it has also unleashed a wave of changes in the...
2       it can be confusing to pick the right smartpho...
3       the mobile application is integrated with a da...
4       we have rounded up some of the gadgets that sh...
                              ...                        
2743    according to researchers, fraud in the mobile ...
2744    the iphone xs and xs max share the apple a12 c...
2745    on the photography front, the note 5 pro featu...
2746    uday mandated that discoms bring the gap betwe...
2747    ripple also helps bank customers send money to...
Name: STORY, Length: 2748, dtype: object

Extracting features from data

In [112]:
#for feature extraction we can use tokenization to splitting up a long text into smaller lines/words then we can use, 
from nltk.tokenize import WordPunctTokenizer # wordPunctTokenizer for extract the tokens from string of words or sentences in the form of alphabetic or nonalphabetic. then,
from sklearn.feature_extraction.text import TfidfVectorizer # we are converting our collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import CountVectorizer # we are transforming a given text into a vector on the basis of the frequency word in text. It is like sparse matrix.

wpt = WordPunctTokenizer() #first we toknizing text
cv = CountVectorizer(tokenizer=wpt.tokenize, stop_words='english', ngram_range=(1,3))#converting to vector

cv.fit(df_train.STORY.tolist() + df_test.STORY.tolist()) 
df_train_cv = cv.transform(df_train.STORY) #transformig it onto train story
df_test_cv = cv.transform(df_test.STORY) #transformig it onto test story

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [113]:
df_train_cv,df_test_cv

(<7628x1039447 sparse matrix of type '<class 'numpy.int64'>'
 	with 1575005 stored elements in Compressed Sparse Row format>,
 <2748x1039447 sparse matrix of type '<class 'numpy.int64'>'
 	with 554993 stored elements in Compressed Sparse Row format>)

In [114]:
#vectorizing 
tf_vector = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 3), use_idf=1.0,smooth_idf=1.0,sublinear_tf=1.0,stop_words = 'english')

tf_vector.fit(df_train.STORY.tolist() + df_test.STORY.tolist())
df_train_tfv = tf_vector.transform(df_train.STORY)
df_test_tfv = tf_vector.transform(df_test.STORY)

Train model


In [115]:
#we are using Logistic regression mltiple for categoring classificaton.
from sklearn.linear_model import LogisticRegression
from scipy.stats.mstats import mode

In [116]:
#traing using logistic regression 
training = {'C': 0.9, 'max_iter': 50, 'multi_class': 'ovr', 'penalty': 'l2'}
lr = LogisticRegression(**training, class_weight='balanced')
lr.fit(df_train_cv, df_train.SECTION)

training = {'C': 250, 'max_iter': 50, 'multi_class': 'ovr', 'penalty': 'l2'}
lr2 = LogisticRegression(**training, class_weight='balanced')
lr2.fit(df_train_tfv, df_train.SECTION)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(C=250, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=50, multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Testing

In [117]:
#testing
lrprediction = lr.predict_proba(df_test_cv)
lrprediction2 = lr2.predict_proba(df_test_tfv)

In [118]:
#printing predicted matrix
lrprediction,lrprediction2

(array([[1.23078009e-08, 9.99999986e-01, 1.50451005e-23, 1.72219540e-09],
        [2.21357423e-02, 5.61302334e-02, 9.11061608e-01, 1.06724159e-02],
        [3.79080507e-08, 9.99999921e-01, 2.62461756e-21, 4.09313654e-08],
        ...,
        [5.13555352e-05, 9.99131674e-01, 6.09595632e-06, 8.10874304e-04],
        [4.60826754e-01, 5.08758417e-03, 1.56772942e-21, 5.34085662e-01],
        [4.40303405e-03, 9.50898942e-01, 1.96483330e-06, 4.46960592e-02]]),
 array([[2.78204123e-05, 9.99961526e-01, 7.39826033e-06, 3.25581245e-06],
        [1.02607302e-02, 3.97042374e-02, 9.48743713e-01, 1.29131915e-03],
        [3.38682378e-05, 9.99884766e-01, 1.84351271e-05, 6.29307546e-05],
        ...,
        [6.71655050e-04, 9.95523082e-01, 1.50450782e-03, 2.30075490e-03],
        [6.94091411e-01, 1.24683122e-02, 1.21774381e-04, 2.93318502e-01],
        [1.05712878e-03, 9.71733421e-01, 1.43403912e-03, 2.57754108e-02]]))

In [119]:
#making predictions and save output in excel file
lrtest = np.mean([lrprediction, lrprediction2], axis=0)
lrpred = np.argmax(lrtest, axis=1)
df = pd.DataFrame()
df['SECTION'] = lrpred.astype(int)
df.to_excel("lrpred.xlsx", index=None)
df.head(10)

,SECTION
0,1
1,2
2,1
3,0
4,1
5,1
6,1
7,2
8,1
9,2
